<a href="https://colab.research.google.com/github/kayvour/Tamil-and-Malayalam-Political-Meme-Classification/blob/main/tamil_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers pytesseract pillow scikit-learn tqdm
!apt-get install tesseract-ocr -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
import os
import torch
import numpy as np
import pandas as pd
import pytesseract
import torch.nn as nn
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset, DataLoader


In [3]:
BASE_PATH = "/content/drive/MyDrive/meme"
IMG_PATH = os.path.join(BASE_PATH, "Train_images")
LABEL_PATH = os.path.join(BASE_PATH, "Train_labels.xlsx")

df = pd.read_excel(LABEL_PATH)

# Remove hidden spaces in column names
df.columns = df.columns.str.strip()

# Create correct image path
df["image_path"] = df["Image_name"].apply(
    lambda x: os.path.join(IMG_PATH, str(x))
)

# Use Level2 as label (most specific class)
df["label"] = df["Level2"]

device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)
print("Total samples:", len(df))

df.head()


Device: cuda
Total samples: 803


,Image_id,Image_name,Level1,Level2,image_path,label
0,0,000.jpg,Troll/Oppose,Troll/Oppose Against Person,/content/drive/MyDrive/meme/Train_images/000.jpg,Troll/Oppose Against Person
1,2,002.jpg,Troll/Oppose,Troll/Oppose Against Person,/content/drive/MyDrive/meme/Train_images/002.jpg,Troll/Oppose Against Person
2,3,003.jpg,Troll/Oppose,Troll/Oppose Against Person,/content/drive/MyDrive/meme/Train_images/003.jpg,Troll/Oppose Against Person
3,4,004.jpg,Troll/Oppose,Troll/Oppose Against Party,/content/drive/MyDrive/meme/Train_images/004.jpg,Troll/Oppose Against Party
4,5,005.jpg,Troll/Oppose,Troll/Oppose Against Person,/content/drive/MyDrive/meme/Train_images/005.jpg,Troll/Oppose Against Person


In [4]:
def extract_text(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        text = pytesseract.image_to_string(img)
        return text.strip()
    except:
        return ""

tqdm.pandas()
df["ocr_text"] = df["image_path"].progress_apply(extract_text)


100%|██████████| 803/803 [16:55<00:00,  1.26s/it]


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

clip_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [ ]:
features = []
labels = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        image = Image.open(row["image_path"]).convert("RGB")
        text = row["ocr_text"]

        inputs = processor(
            text=[text],
            images=image,
            return_tensors="pt",
            padding=True
        ).to(device)

        with torch.no_grad():
            outputs = clip_model(**inputs)

        image_emb = outputs.image_embeds
        text_emb = outputs.text_embeds

        # Normalize
        image_emb = image_emb / image_emb.norm(dim=-1, keepdim=True)
        text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

        combined = torch.cat([image_emb, text_emb], dim=1)

        features.append(combined.cpu().numpy()[0])
        labels.append(row["label"])

    except:
        continue

features = np.array(features)
labels = np.array(labels)

print("Feature shape:", features.shape)
print("Label shape:", labels.shape)


In [7]:
le = LabelEncoder()
labels = le.fit_transform(labels)

print("Classes:", le.classes_)


Classes: ['Support for party' 'Support for person' 'Troll/Oppose Against Party'
 'Troll/Oppose Against Person']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

train_X = torch.tensor(X_train, dtype=torch.float32)
train_y = torch.tensor(y_train, dtype=torch.long)

test_X = torch.tensor(X_test, dtype=torch.float32)
test_y = torch.tensor(y_test, dtype=torch.long)

print(train_X.shape)
print(test_X.shape)


In [9]:
class MemeDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = MemeDataset(train_X, train_y)
test_dataset = MemeDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [10]:
class MemeClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = MemeClassifier(1024, len(le.classes_))


In [11]:
class_counts = torch.bincount(train_y)
class_weights = 1.0 / class_counts.float()
class_weights = class_weights / class_weights.sum()
class_weights = class_weights * len(le.classes_)

print("Class counts:", class_counts)
print("Class weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)


Class counts: tensor([ 17,  57, 101, 401])
Class weights: tensor([2.6508, 0.7906, 0.4462, 0.1124])


In [12]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=0.01
)


In [13]:
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == y_batch).sum().item()

    acc = correct / len(train_dataset)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Loss: {total_loss/len(train_loader):.4f}")
    print(f"Train Accuracy: {acc:.4f}")
    print("-"*40)


Epoch 1/20
Loss: 1.3908
Train Accuracy: 0.2413
----------------------------------------
Epoch 2/20
Loss: 1.0372
Train Accuracy: 0.4323
----------------------------------------
Epoch 3/20
Loss: 0.8377
Train Accuracy: 0.6024
----------------------------------------
Epoch 4/20
Loss: 0.6310
Train Accuracy: 0.7014
----------------------------------------
Epoch 5/20
Loss: 0.5143
Train Accuracy: 0.7500
----------------------------------------
Epoch 6/20
Loss: 0.4001
Train Accuracy: 0.8038
----------------------------------------
Epoch 7/20
Loss: 0.3400
Train Accuracy: 0.8576
----------------------------------------
Epoch 8/20
Loss: 0.2765
Train Accuracy: 0.9080
----------------------------------------
Epoch 9/20
Loss: 0.2036
Train Accuracy: 0.9253
----------------------------------------
Epoch 10/20
Loss: 0.1636
Train Accuracy: 0.9479
----------------------------------------
Epoch 11/20
Loss: 0.1352
Train Accuracy: 0.9688
----------------------------------------
Epoch 12/20
Loss: 0.1235
Train

In [14]:
model.eval()
all_preds = []

with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.numpy())

print(classification_report(test_y.numpy(), all_preds))


              precision    recall  f1-score   support

           0       0.14      0.20      0.17         5
           1       0.45      0.64      0.53        14
           2       0.27      0.24      0.26        25
           3       0.79      0.75      0.77       100

    accuracy                           0.63       144
   macro avg       0.41      0.46      0.43       144
weighted avg       0.64      0.63      0.64       144

